In [2]:
from model.yolov2 import YOLOv2, darknet19
import keras.backend as K
from cfg import *

K.clear_session() # Avoid duplicate model
darknet19 = darknet19(freeze_layers=True)
yolov2    = YOLOv2(feature_extractor=darknet19, num_anchors=len(ANCHORS), num_classes=N_CLASSES)
model     = yolov2.model
print(model.summary())

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 128         conv2d_1[0][0]                   
____________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [3]:
from keras.models import load_model

model.load_weights('yolov2.weights', by_name=True)

In [4]:

THRESHOLD = 0.3
LABELS   =  ['addedLane', 'bicyclesMayUseFullLane', 'curveLeft', 'curveRight', 'doNotEnter', 'intersection', 'intersectionLaneControl', 'keepRight', 'laneEnds',
 'leftAndUTurnControl', 'merge', 'noLeftAndUTurn', 'noParking', 'noRightTurn',
 'noUTurn', 'pedestrianCrossing', 'school', 'signalAhead', 'speedBumpsAhead',
 'speedLimit15', 'speedLimit25', 'speedLimit30', 'speedLimit35', 'speedLimit40',
 'speedLimit45', 'speedLimit50', 'speedLimit60', 'stop', 'stopAhead',
 'yieldAhead', 'yieldToPedestrian']

In [5]:
import cv2

class BoundBox:
    def __init__(self, class_num):
        self.x, self.y, self.w, self.h, self.c = 0., 0., 0., 0., 0.
        self.probs = np.zeros((class_num,))
    def iou(self, box):
        intersection = self.intersect(box)
        union = self.w*self.h + box.w*box.h - intersection
        return intersection/union
        
    def intersect(self, box):
        width  = self.__overlap([self.x-self.w/2, self.x+self.w/2], [box.x-box.w/2, box.x+box.w/2])
        height = self.__overlap([self.y-self.h/2, self.y+self.h/2], [box.y-box.h/2, box.y+box.h/2])
        return width * height
        
    def __overlap(self, interval_a, interval_b):
        x1, x2 = interval_a
        x3, x4 = interval_b
        if x3 < x1:
            if x4 < x1:
                return 0
            else:
                return min(x2,x4) - x1
        else:
            if x2 < x3:
                return 0
            else:
                return min(x2,x4) - x1


In [6]:
import tensorflow as tf 


def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def sigmoid(x):
    return 1. / (1.  + np.exp(-x))

def interpret_netout(image, netout):
    output_shape = K.shape(netout)
    boxes = []
    with sess.as_default():
        GRID_W = tf.cast(output_shape[0], tf.int32).eval()
        GRID_H = tf.cast(output_shape[1], tf.int32).eval()
    
    netout = np.reshape(netout, [GRID_W, GRID_H, len(ANCHORS), -1])
    # interpret the output by the network
    print(GRID_W, GRID_H)
    print(np.shape(netout))
    for row in range(GRID_W):
        for col in range(GRID_H):
            for b in range(len(ANCHORS)):
                box = BoundBox(N_CLASSES)

                # first 5 weights for x, y, w, h and confidence
                box.x, box.y, box.w, box.h, box.c = netout[row, col, b,:5]
                box.x = (col + sigmoid(box.x)) / GRID_W
                box.y = (row + sigmoid(box.y)) / GRID_H
#                 box.w = ANCHORS[b][0] * np.exp(box.w) / GRID_W
#                 box.h = ANCHORS[b][1] * np.exp(box.h) / GRID_H
                box.w = ANCHORS[b][0] * np.exp(box.w)
                box.h = ANCHORS[b][1] * np.exp(box.h)
                box.c = sigmoid(box.c)

                # last  weights for class probabilities
                classes = netout[row,col,b,5:]
                box.probs = softmax(classes) * box.c # P(obj|class) = P(obj)*softmax(P(classes))
                box.probs *= box.probs > THRESHOLD
                boxes.append(box)

    # suppress non-maximal boxes
    for c in range(N_CLASSES):
        sorted_indices = list(reversed(np.argsort([box.probs[c] for box in boxes])))

        for i in xrange(len(sorted_indices)):
            index_i = sorted_indices[i]
            
            if boxes[index_i].probs[c] == 0: 
                continue
            else:
                for j in xrange(i+1, len(sorted_indices)):
                    index_j = sorted_indices[j]
                    
                    if boxes[index_i].iou(boxes[index_j]) >= 0.4:
                        boxes[index_j].probs[c] = 0

    # draw the boxes using a threshold
    for box in boxes:
        max_indx = np.argmax(box.probs)
        max_prob = box.probs[max_indx]
        if max_prob > THRESHOLD:
            xmin  = int((box.x - box.w/2) * image.shape[1])
            xmax  = int((box.x + box.w/2) * image.shape[1])
            ymin  = int((box.y - box.h/2) * image.shape[0])
            ymax  = int((box.y + box.h/2) * image.shape[0])
            print((xmin,ymin), (xmax,ymax))
            print(max_indx)
            cv2.rectangle(image, (xmin,ymin), (xmax,ymax),np.random.randint(0, 255, [3]), 3)
            cv2.putText(image, LABELS[max_indx], (xmin, ymin - 12), 0, 1e-3 * image.shape[0], (0,255,0), 2)
    return image

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
image = cv2.imread('stop.jpg')

plt.figure(figsize=(10,10))

input_image = cv2.resize(image, (960, 1280))
input_image = input_image / 255.
input_image = input_image[:,:,::-1]
input_image = np.expand_dims(input_image, 0)

sess = K.get_session()

netout = model.predict(input_image)

#print netout
image = interpret_netout(image, netout[0])
plt.imshow(image[:,:,::-1]); plt.show()